In [28]:
import os
import nltk
import json
import re
# Import stopwords database from the nltk corpus 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Define stopwords
stop_words = set(stopwords.words('english'))

In [29]:
def read_files_in_directory(directory):
    code_files_content = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.py', '.ipynb', '.md')):  # Add more file extensions as needed
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    code_files_content.append(f.read())
    return code_files_content

In [30]:
def extract_comments_and_code(text):
    comments = []
    code = []
    for line in text.split('\n'):
        line = line.strip()
        # Assuming single-line comments for simplicity; adjust regex for multi-line comments
        if line.startswith('#') or line.startswith('//') or re.match(r'^\s*/\*.*\*/\s*$', line):
            comments.append(line)
        else:
            code.append(line)
    return ' '.join(comments), ' '.join(code)

def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens

In [31]:
# Path to your code directory
code_directory = 'C:\\Users\\Lardex\\Desktop\\DU Assginments\\AI_BOOTCAMP\\DU-VIRT-AI-PT-10-2023-U-LOLC\\01-Lesson-Plans\\02-Python-Programming-1'
# code_directory = 'C:\\Users\\Lardex\\Desktop\\DU Assginments\\AI_BOOTCAMP\\DU-VIRT-AI-PT-10-2023-U-LOLC'
code_files_content = read_files_in_directory(code_directory)

In [32]:
processed_data = {"files": []}

for code in code_files_content:
    comments, code_only = extract_comments_and_code(code)
    tokenized_comments = tokenize_and_remove_stopwords(comments)
    tokenized_code = word_tokenize(code_only)  # Do not remove stopwords from code
    file_data = {
        "comments": tokenized_comments,
        "code": tokenized_code
    }
    processed_data["files"].append(file_data)

print(processed_data)

{'files': [{'comments': ['#', 'Instructor', 'Demo'], 'code': ['--', '-', '©', '2023', 'edX', 'Boot', 'Camps', 'LLC', '.', 'Confidential', 'and', 'Proprietary', '.', 'All', 'Rights', 'Reserved', '.']}, {'comments': ['#', 'Pseudocode', 'Putting', 'Dishes', 'Away', '#', 'Tasks', ':', '#', 'Stack', 'plain', 'plates', '#', 'Stack', 'fancy', 'plates', '#', 'Stack', 'plain', 'bowls', '#', 'Pattern', 'recognition', ':', '#', 'Plain', 'plates', 'go', 'bottom', 'shelf', '#', 'Fancy', 'plates', 'go', 'top', 'shelf', '#', 'Bowls', 'go', 'top', 'plain', 'plates', '#', 'Abstraction', ':', '#', 'Ignore', 'knife', '#', 'Ignore', 'towel', '#', 'Sequence', ':', '#', '1', '.', 'Event', '1', ':', 'Put', 'away', 'plates', ':', '#', '2', '.', 'Event', '2', ':', 'Put', 'away', 'bowls', ':', '#', 'Sequence', ':', '#', '1', '.', 'Event', '1', ':', 'Put', 'away', 'plates', ':', '#', 'Conditional', ':', '(', 'plate', 'fancy', ')', ':', '#', 'stack', 'top', 'shelf', '#', 'Else', ':', '#', 'stack', 'bottom', 'shel

In [33]:
with open('processed_code.json', 'w', encoding='utf-8') as json_file:
    json.dump(processed_data, json_file, ensure_ascii=False, indent=4)